In [ ]:
%run ~/common_init.py
import flamedisx as fd
import tensorflow as tf
#fd.utils.FLOAT_TYPE = tf.float64
fd.float_type()

In [ ]:
rn_data = pd.read_csv('data_Rn220_lowenergy.csv')[::32]
fd.SR0ERSource.annotate_data(rn_data)
rn_data = rn_data[rn_data['e_vis'] < 10].copy()
rn_data = rn_data[['s1', 's2'] + ['x_observed', 'y_observed', 'x', 'y', 'z', 'drift_time']]

In [ ]:
len(rn_data)

In [ ]:
ll = fd.LogLikelihood(
    sources=dict(er=fd.SR0ERSource),
    batch_size=1,  # For CPU, batching does not help
    free_rates='er',
    data=rn_data,
    elife=(300e3, 500e3, 3),
    erqy_a=(5., 25., 2),
    erqy_b=(-29., -26., 2),
    erqy_c=(31.9, 32.8, 2),
    g2=(25., 35., 2),
    n_trials=int(2e5))
len(rn_data), len(rn_data) / ll.mu_itps['er']().numpy()

In [ ]:
for pname in ll.param_names:
    if pname.endswith('_rate_multiplier'):
        continue
    start, stop, n = ll.param_specs[pname]
    x = np.linspace(start, stop, 100, dtype=np.float32)
    y = np.array([ll.mu_itps['er'](**{pname: tf.constant(q)}) for q in x])
    plt.plot(x, y)
    plt.title(pname)
    plt.show()

In [ ]:
guess = ll.guess().numpy()
guess[0] = 0.02 #* 32

In [ ]:
%%time
bestfit = ll.bestfit(guess)

In [ ]:
%%time
inv_hess = ll.inverse_hessian(bestfit)

In [ ]:
ll.summary(bestfit, inverse_hessian=inv_hess)

In [ ]:
n_trials = int(1e5)

sim_guess = fd.SR0ERSource.simulate(
    n_trials, 
    data=rn_data,
    **ll.params_to_dict(guess))

sim_bf = fd.SR0ERSource.simulate(
    n_trials, 
    data=rn_data,
    **ll.params_to_dict(bestfit.numpy()))

bins = np.linspace(0, 11, 10)
for d_sim in sim_guess, sim_bf:
    Histdd(d_sim, dimensions=(
        ('e_vis', np.linspace(0, 11, 50)),
        ('electron_produced_mle', np.linspace(0, 350, 50)),
    )).plot()
    d = rn_data
    plt.scatter(d['e_vis'], d['electron_produced_mle'], c='white', s=5)
    plt.show()
